In [ ]:
from datetime import datetime
from traitlets import link, dlink
from ipywidgets import Text, VBox, HTML
from dateslider.widgets import DateSlider
from dateutil import parser

from math import pi
import pandas as pd

from bokeh.sampledata.stocks import MSFT
from bokeh.plotting import figure, output_notebook, show, ColumnDataSource
from bokeh.models import Span
from bokeh.io import push_notebook
output_notebook()

In [ ]:
df = pd.DataFrame(MSFT)[:50]
df['date'] = pd.to_datetime(df['date'])

mids = (df.open + df.close)/2
spans = abs(df.close-df.open)

inc = df.close > df.open
dec = df.open > df.close
w = 12*60*60*1000 # half day in ms

In [ ]:
p = figure(x_axis_type="datetime", plot_width=1000)
p.title = "MSFT Candlestick"
p.xaxis.major_label_orientation = pi/4
p.grid.grid_line_alpha=0.3
p.segment(df.date, df.high, df.date, df.low, color='black')
p.rect(df.date[inc], mids[inc], w, spans[inc], fill_color="#D5E1DD", line_color="black")
p.rect(df.date[dec], mids[dec], w, spans[dec], fill_color="#F2583E", line_color="black")

source = ColumnDataSource({"location": [datetime.now().timestamp() * 1e3]})
line = Span(
    location=df.date.min().to_datetime().timestamp() * 1e3,
    dimension='height', line_color='green',
    line_dash='dashed')
p.renderers += [line]

In [ ]:
ds = DateSlider(
    value=datetime.now(),
    start=df.date.min().to_datetime(),
    end=df.date.max().to_datetime()
)

@ds.observe
def _ds_changed(change):
    line.location = ds.value.timestamp() * 1000
    if line in p.renderers:
        p.renderers.remove(line)
        push_notebook()
    p.renderers.append(line)
    push_notebook()

In [ ]:
show(p);

In [ ]:
ds